### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent
    movie_mat = np.random.rand(n_movies, latent_features)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if (ratings_mat[i, j] > 0):
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum = diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2 * diff * movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2 * diff * user_mat[i, k])
        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=10) # use your function with 3 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 3.599114
2 		 2.786453
3 		 1.781811
4 		 0.906972
5 		 0.384601
6 		 0.153426
7 		 0.065871
8 		 0.033301
9 		 0.020327
10 		 0.014619


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  7.64035159   9.24875141   8.43124297]
 [  6.83101693   8.71447774   7.82628094]
 [  7.40665458   8.87916998   8.1539758 ]
 [  8.50763862  10.43981777   9.44711638]
 [  7.82897049   9.71035277   8.74132282]
 [  7.72011251   9.32875446   8.58049223]]
[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


In [7]:
print("The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. Clearly the model is not done learning, but things are looking good.")

The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. Clearly the model is not done learning, but things are looking good.


_**Task: compare the predicted vs actual ratings and write your findings here**_

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 3 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 300.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about the `mean squared error` at the end of each training iteration?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=300) #use your function with 3 latent features, lr of 0.005 and 300 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 2.798394
2 		 1.815222
3 		 0.922855
4 		 0.374356
5 		 0.136574
6 		 0.053176
7 		 0.025353
8 		 0.015443
9 		 0.011500
10 		 0.009744
11 		 0.008872
12 		 0.008387
13 		 0.008082
14 		 0.007866
15 		 0.007697
16 		 0.007554
17 		 0.007425
18 		 0.007304
19 		 0.007189
20 		 0.007076
21 		 0.006965
22 		 0.006854
23 		 0.006743
24 		 0.006632
25 		 0.006519
26 		 0.006406
27 		 0.006291
28 		 0.006175
29 		 0.006058
30 		 0.005939
31 		 0.005819
32 		 0.005698
33 		 0.005576
34 		 0.005453
35 		 0.005329
36 		 0.005205
37 		 0.005079
38 		 0.004953
39 		 0.004827
40 		 0.004701
41 		 0.004574
42 		 0.004448
43 		 0.004322
44 		 0.004197
45 		 0.004072
46 		 0.003948
47 		 0.003825
48 		 0.003703
49 		 0.003583
50 		 0.003464
51 		 0.003347
52 		 0.003232
53 		 0.003119
54 		 0.003009
55 		 0.002900
56 		 0.002795
57 		 0.002692
58 		 0.002591
59 		 0.002494
60 		 0.002399
61 		 0.002307
62 		 0.002219
63 		 0.002133
64 		 0

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  7.04202748  10.014702     7.95693585]
 [  5.97769952   9.99178023   7.02255479]
 [  7.9629939    8.98670411   8.03786338]
 [  8.01136056  10.00448325   9.9883182 ]
 [  9.00002902   8.99979557   9.00019455]
 [  8.00396786   9.0015531    8.99587791]]
[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


In [8]:
print("In this case, we were able to completely reconstruct the item-movie matrix to closer to 0 mean squared error at the end of 300th iterations.")

In this case, we were able to completely reconstruct the item-movie matrix to closer to 0 mean squared error at the end of 300th iterations.


_**Task: compare the predicted vs actual ratings and write your findings here**_

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [9]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,   8.],
        [  6.,  10.,   7.],
        [  8.,   9.,   8.],
        [  8.,  10.,  10.],
        [  9.,   9.,   9.],
        [  8.,   9.,   9.]])

In [10]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=450) #use your function with 3 latent features, lr of 0.005 and 450 iteration

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 3.400742
2 		 2.405321
3 		 1.361464
4 		 0.604577
5 		 0.225375
6 		 0.081617
7 		 0.034019
8 		 0.018349
9 		 0.012910
10 		 0.010994
11 		 0.010412
12 		 0.010355
13 		 0.010491
14 		 0.010677
15 		 0.010851
16 		 0.010990
17 		 0.011087
18 		 0.011144
19 		 0.011165
20 		 0.011157
21 		 0.011123
22 		 0.011070
23 		 0.011000
24 		 0.010916
25 		 0.010822
26 		 0.010718
27 		 0.010608
28 		 0.010491
29 		 0.010369
30 		 0.010242
31 		 0.010112
32 		 0.009978
33 		 0.009842
34 		 0.009702
35 		 0.009560
36 		 0.009416
37 		 0.009269
38 		 0.009121
39 		 0.008971
40 		 0.008819
41 		 0.008665
42 		 0.008510
43 		 0.008354
44 		 0.008196
45 		 0.008038
46 		 0.007879
47 		 0.007718
48 		 0.007558
49 		 0.007397
50 		 0.007235
51 		 0.007074
52 		 0.006912
53 		 0.006751
54 		 0.006590
55 		 0.006430
56 		 0.006270
57 		 0.006112
58 		 0.005955
59 		 0.005799
60 		 0.005644
61 		 0.005491
62 		 0.005340
63 		 0.005191
64 		 0

In [11]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 7.338749496159472

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [13]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=3, learning_rate=0.005, iters=500) #fit to 1000 users with 3 latent features, lr of 0.005, and 500 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 


IndexError: index 3336 is out of bounds for axis 1 with size 3

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [ ]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")